In [65]:
import pandas as pd
import numpy as np

In [66]:
# Import raw data
data = 'test'
raw_df = pd.read_csv(f'../rawData/{data}.csv') 
clean_df = raw_df.copy()

In [67]:
# Prepare to map ordinals
mapping1 = {np.nan:0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
mapping2 = {np.nan:0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4}
mapping3 = {np.nan:0, 'IR3':1, 'IR2':2, 'IR1':3, 'Reg':4}
mapping4 = {np.nan:0, 'Lvl':1, 'Bnk':2, 'HLS':3, 'Low':4}
mapping5 = {np.nan:0, 'ELO':1, 'NoSeWa':2, 'NoSewr':3, 'AllPub':4}
mapping6 = {'Sev':1, 'Mod':2, 'Gtl':3}
#mapping7 = {np.nan:0, '1Story':1, '1.5Unf':2, '1.5Fin':3, '2Story':4, '2.5Unf':5, '2.5Fin':6,
#           'SFoyer':7, 'SLvl':8}
mapping8 = {np.nan:0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}
mapping9 = {np.nan:0, 'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6,
            'Min1':7, 'Typ':8}
mapping10 = {np.nan:0, 'Unf':1, 'RFn':2, 'Fin':3}
mapping11 = {np.nan:0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}

In [68]:
# Map ordinals
clean_df['LotShape'] = [mapping3[val] for val in raw_df['LotShape']]
clean_df['LandContour'] = [mapping4[val] for val in raw_df['LandContour']]
clean_df['Utilities'] = [mapping5[val] for val in raw_df['Utilities']]
clean_df['BsmtExposure'] = [mapping2[val] for val in raw_df['BsmtExposure']]
clean_df['Functional'] = [mapping9[val] for val in raw_df['Functional']]
clean_df['GarageFinish'] = [mapping10[val] for val in raw_df['GarageFinish']]
clean_df['Fence'] = [mapping11[val] for val in raw_df['Fence']]
clean_df['LandSlope'] = [mapping6[val] for val in raw_df['LandSlope']]
#clean_df['HouseStyle'] = [mapping7[val] for val in raw_df['HouseStyle']]

tmpCol = ['ExterQual', 'BsmtQual', 'BsmtCond', 'ExterCond', 'BsmtQual', 'BsmtCond',
         'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond',
         'PoolQC']
for col in tmpCol :
    try:
        clean_df[col] = [mapping1[val] for val in raw_df[col]]
    except:
        print(col)

tmpCol = ['BsmtFinType1', 'BsmtFinType2']
for col in tmpCol :
    clean_df[col] = [mapping8[val] for val in raw_df[col]]

In [69]:
# Fill missing values in Lot Frontage

dict_neigh = raw_df.groupby(by = 'Neighborhood').agg({'LotFrontage':'median'}).to_dict()
clean_df.loc[clean_df['LotFrontage'].isna(),  'LotFrontage'] = \
            list(map(lambda n : dict_neigh['LotFrontage'][n], \
            raw_df.loc[raw_df['LotFrontage'].isna()]['Neighborhood']))

In [70]:
# Drop quasi-empty columns
clean_df = clean_df.drop(columns = ['PoolQC', 'MiscFeature', 'Alley'])

In [71]:
# MasVrnArea is na when MasVnrType is also NA.  
# Probably best to zero the float and 'None' the string
clean_df.loc[clean_df['MasVnrArea'].isna(), 'MasVnrArea'] = 0
clean_df.loc[clean_df['MasVnrType'].isna(), 'MasVnrType'] = 'None'

In [72]:
# There's only one missing value in Electrical
# Let's replace that with the modal value
clean_df.loc[clean_df['Electrical'].isna(), 'Electrical'] = 'SBrkr'

In [73]:
# GarageYrBlt has NA when GarageType = 'NA'.  Let's make the nominal variable 'none'
clean_df.loc[clean_df['GarageType'].isna(), 'GarageType'] = 'None'
# For the ordinal, we could encode that as 0 but this will lead to a wide data range 
# (0s and 1985 for example). I think it might be better to define it as equal to 
# house built date to avoid this
clean_df.loc[clean_df['GarageYrBlt'].isna(), 'GarageYrBlt'] = \
            clean_df.loc[clean_df['GarageYrBlt'].isna(), 'YearBuilt']

In [74]:
miss_cols = clean_df.columns[clean_df.isna().any()]
miss_cols

Index(['MSZoning', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
       'GarageCars', 'GarageArea', 'SaleType'],
      dtype='object')

In [75]:
# Clean up any stragglers (needed for the test set) by inserting modal values
for col in miss_cols:
    miss_rows = clean_df[col].isna()
    mode = clean_df.loc[miss_rows==False, col].value_counts().index[0]
    print('{0}: Missing {1}, modal value {2}'.format(col, miss_rows.sum(), mode))
    clean_df.loc[miss_rows, col] = mode

MSZoning: Missing 4, modal value RL
Exterior1st: Missing 1, modal value VinylSd
Exterior2nd: Missing 1, modal value VinylSd
BsmtFinSF1: Missing 1, modal value 0.0
BsmtFinSF2: Missing 1, modal value 0.0
BsmtUnfSF: Missing 1, modal value 0.0
TotalBsmtSF: Missing 1, modal value 0.0
BsmtFullBath: Missing 2, modal value 0.0
BsmtHalfBath: Missing 2, modal value 0.0
GarageCars: Missing 1, modal value 2.0
GarageArea: Missing 1, modal value 0.0
SaleType: Missing 1, modal value WD


In [76]:
clean_df.to_csv(f'./{data}_cleaned.csv', index=False)